In [49]:
import pandas as pd
from collections import Counter
import numpy as np

# base_df = pd.read_csv("fake_data.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
base_df = pd.read_csv("../BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",
                usecols=['CustomerID', 'AgeCategory', 'Gender', 'Office_U', 'Office_W',
       'SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info'])
base_df.rename(columns={'URL_FILE':'Activity'}, inplace=True)
base_df["TIMESTAMP"] = pd.to_datetime(base_df["TIMESTAMP"], infer_datetime_format=True)
base_df = base_df.sort_values(["SessionID", "TIMESTAMP"])

C:\Users\krdeg\AppData\Local\Temp\ipykernel_10620\568469614.py:6: DtypeWarning: Columns (11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv("../BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",


In [50]:
# Helper functions
def create_df_for_pivot(df: pd.DataFrame):
  df = df.copy()
  #Create consecutive column with the consecutive activity and count how many time each pair occurs
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

  #Count all the pairs and safe in new DataFrame
  df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

  df_for_pivot_1.fillna("end_session", inplace=True)
  return df_for_pivot_1
  
def make_pivot(df, index_names, column_names):
  df = df.copy()
  """
  Functions that takes in a DataFrame and returns a pivot table with all the chances
  
  Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
  """
  df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
  
  #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
  df_chances["total_row_count"] = df_chances.sum(axis=1)
  df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
  
  df_chances.fillna(0, inplace=True)
  df_chances.drop("total_row_count", axis=1, inplace=True)

  return df_chances

  
# Create df for pivot
df_for_pivot = create_df_for_pivot(base_df)
df_chances_1 = make_pivot(df_for_pivot, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = base_df.groupby("SessionID").nth(0)["Activity"].value_counts() / base_df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

df_chances_total = df_chances_1 
df_chances_total

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
final_df = df_chances_total.iloc[:, :-1]
final_df

level_1,/,//,//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,//werk_nl/werknemer/contact,//werk_nl/werknemer/contact/email_uw_vraag,//werk_nl/werknemer/contact/vestiging-zoeken,//werk_nl/werknemer/home,//werk_nl/werknemer/over-werk-nl/handleiding,//werk_nl/werknemer/over-werk-nl/handleiding/cv-plaatsen,//werk_nl/werknemer/over-werk-nl/handleiding/sollicitatie-wijziging,...,/xpsitem/wdo_013389,/xpsitem/wdo_013407,/xpsitem/wdo_013827,/xpsitem/wdo_014041,/zoeken_portlet/ajax/addVacatureToGereageerd,/zoeken_portlet/ajax/zoekAantalIndicatief,/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet_wg/ajax/selectContactpersoon,/zoeken_portlet_wg/ajax/selectVacature,end_session
level_0,,,,,,,,,,,,,,,,,,,,,
/,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.073834
//,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.035714
//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact/email_uw_vraag,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/zoeken_portlet/ajax/addVacatureToGereageerd,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017588,0.002513,0.005025,0.0,0.0,0.055276
/zoeken_portlet/ajax/zoekAantalIndicatief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000107,0.537184,0.051269,0.0,0.0,0.007539
/zoeken_portlet/ajax/zoekBeroep,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.441592,0.489910,0.0,0.0,0.003979


In [51]:
def skipping_events(prop_matrix:pd.DataFrame, start:str,skip:str,end:list[str],deviation_rate:int) -> pd.DataFrame:
  df = prop_matrix.copy()
  deviation_rate = deviation_rate / 100
  start_to_skip = df.loc[start,skip]
  new_start_to_skip = start_to_skip * (1 - deviation_rate)
  df.loc[start,skip] = new_start_to_skip
  increase = (start_to_skip - new_start_to_skip) / len(end)

  for event in end:
    df.loc[start,event] += increase
      
  return df

In [52]:
patterns_in_data = pd.read_csv('./gen_patterns/skip_df_15.csv')
# create a list of all the events that should be skipped

patters_df_ready = patterns_in_data.copy()
patters_df_ready = patters_df_ready.groupby(['Activity_0', 'Activity_1'],as_index=False)['Activity_2'].agg(lambda x: list(x))
patters_df_ready['len-c'] = patters_df_ready['Activity_2'].apply(lambda x: len(x))
patterns_to_generate = patters_df_ready.copy()

patterns_to_generate

,Activity_0,Activity_1,Activity_2,len-c
0,/portal/page/portal/home/diensten/overzicht,/portal/page/portal/home/diensten/aanvragen-ww,[/werk_nl/werknemer/home],1
1,/portal/page/portal/werk_nl/werknemer/eintake/...,/werk_nl/werknemer/eintake/inschrijven,[/portal/page/portal/home/diensten/aanvragen-ww],1
2,/shared/foutopgetreden.html,/werk_nl/werknemer/uitkering-aanvragen,[/portal/page/portal/werk_nl/werknemer/uitkeri...,1
3,/werk_nl/werknemer/eintake/inschrijven,/portal/page/portal/home/diensten/aanvragen-ww,[/portal/page/portal/home/diensten/overzicht],1
4,/werk_nl/werknemer/home,/werk_nl/werknemer/uitkering-aanvragen,[/portal/page/portal/werk_nl/werknemer/uitkeri...,1
5,/werk_nl/werknemer/home,/werk_nl/werknemer/vacatures,"[/zoeken_portlet/ajax/zoekBeroep, /zoeken_port...",2
6,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,"[/ewerkmap/readdetails, /werk_nl/werknemer/mij...",3
7,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...,[/werk_nl/ignore_request_page],1
8,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/home,[/werk_nl/werknemer/over-werk-nl/evenementen/e...,1
9,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...,[/zoeken_portlet/ajax/zoekBeroep],1


In [53]:
# deviation_rate = 100

In [54]:
def skip_mul_events(_df:pd.DataFrame,patt_df:pd.DataFrame,deviation_rate:int):
  df = _df.copy()
  patt_df = patt_df.copy()
  for index, row in patt_df.iterrows():
    df = skipping_events(df, row['Activity_0'], row['Activity_1'], row['Activity_2'], deviation_rate)
  return df

In [55]:
# skipped_df = skip_mul_events(final_df, patterns_to_generate, 20)

In [56]:
def gen_markov_chain(amount:int,start_chances,df_dict:pd.DataFrame, patterns_df:pd.DataFrame) -> list:
  df_dict = df_dict.copy()
  generated_sessions = []
  while len(generated_sessions) < amount:
    flag = False
    new_session = []
    # choose start activity
    start_activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
    new_session.append(start_activity[0])
    
    while new_session[-1] != 'end_session':
      curr_activity = new_session[-1]
      row_activity = df_dict.loc[curr_activity,:]
      new_activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values))[0]
      new_session.append(new_activity)
      # only check if pattern is generated while the flag is false
      if flag == False and ((patterns_df['Activity_0'] == curr_activity) & (patterns_df['Activity_2'] == new_activity)).any():
        flag = True
    
    if flag: 
      generated_sessions.append(new_session)
      if len(generated_sessions)  % 50 == 0 : print(len(generated_sessions))

  return generated_sessions

In [57]:
# ress,pat = gen_markov_chain(1000,start_chances,skipped_df,patterns_in_data)

In [58]:
def markov_result_to_df(markov_result, name_index, name_csv):
    activity_list_final = []
    session_id_list = []
    for nbr in range(len(markov_result)):
        string_generated = name_index + str(nbr)
        for i in markov_result[nbr]:
            if i != "end_session":
                activity_list_final.append(i)
                session_id_list.append(string_generated)
                
    df_generated = pd.DataFrame(list(zip(session_id_list, activity_list_final)),
               columns =['SessionID', 'URL_FILE'])
    df_generated.to_csv(name_csv,index=False)
    
# markov_result_to_df(markov_result, "gen_session_", "gen_sessions/skipped/generated_withCircle_top3_50%_2500.csv")

In [59]:
deviation_rates = [5,10,25,50,75,100]

In [60]:
def gen_all_sessions(rates:list[int],amount:int,base_df=final_df, start_chances_df=start_chances,generation_df=patterns_to_generate,patterns_in_data=patterns_in_data):
  for rate in rates:
    print(f'start with rate: {rate}')
    print(f'Generate the prob matrix for rate: {rate}')
    new_df = base_df.copy()
    prob_matrix = skip_mul_events(new_df,generation_df,rate)
    print(f'Generate sessions rate: {rate}')
    sessions = gen_markov_chain(amount,start_chances_df,prob_matrix,patterns_in_data)
    print(f'Sessions Generated')
    markov_result_to_df(sessions,f'rate_{rate}_',f'gen_sessions/{rate}_{amount}.csv')
    print(f'done with {rate}')
    

In [61]:
gen_all_sessions(deviation_rates,1000)

start with rate: 5
Generate the prob matrix for rate: 5
Generate sessions rate: 5
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 5
start with rate: 10
Generate the prob matrix for rate: 10
Generate sessions rate: 10
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 10
start with rate: 25
Generate the prob matrix for rate: 25
Generate sessions rate: 25
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 25
start with rate: 50
Generate the prob matrix for rate: 50
Generate sessions rate: 50
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 50
start with rate: 75
Generate the prob matrix for rate: 75
Generate sessions rate: 75
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Sessions Generated
done with 75
start with rate: 10